# Setup Model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from UtilityMethods import utils
import sys
#import gym
import pickle
import time
import pulp as p
import math
from copy import copy
import pprint as pp
import itertools
from tqdm import tqdm

## Global variables

In [2]:
# Global variables

IS_VISIT_DEPENDENT = False # whether the above empirical estimates are visit-dependent or not
DATA = '../../../../Codes/Accord/data/ACCORD_BPClass_v2.csv'

EPISODE_LENGTH = 40 # average number of visits per patient
CONSTRAINT = 800 # 20 deviation * 40 visits 
C_b = 20  # change this if you want different baseline policy.

NUMBER_EPISODES = 1e4
NUMBER_SIMULATIONS = 1

delta = 0.01 # bound

EPS = 0.01 # not used
M = 0 # not used

## State space and action space

In [ ]:
# state space, actions available in each state are always the same
"""
state_features = ['sbp_discrete','hba1c_discrete','TC_discrete','hdl_discrete','BMI_discrete'] 
fea1 = ['0', '1', '2', '3'] # possible values for sbp_discrete
fea2 = ['0', '1', '2', '3', '4', '5', '6', '7']
fea3 = ['0', '1', '2', '3']
fea4 = ['0', '1', '2', '3']
fea5 = ['0', '1', '2', '3']
"""

state_features = ['sbp_discrete', 'hba1c_discrete'] 
fea1 = ['0', '1', '2'] # possible values for sbp_discrete, merge 3 with 2
fea2 = ['0', '1', '2', '3'] # merge every 2 adjacent levels into 1 level
fea3 = ['0', '1', '2'] # possible values for TC_discrete, merge 3 with 2
fea4 = ['0', '1', '2'] # merge hdl_discrete 3 with 2

combinations = itertools.product(fea1, fea2)
states = [''.join(i) for i in combinations]
print('len(states) =', len(states))
print(states[:5])

N_STATES = len(states) # number of states = 2048
state_code_to_index = {code: i for i, code in enumerate(states)}
# print the first 5 state_code_to_index
for i in range(3):
    print(states[i], state_code_to_index[states[i]])
print()

# action space, 000000000 means bpclass_none, 111111111 means all bpmed class are precribed
"""
action_features = ['Diur', 'ACE', 'Beta-blocker', 'CCB', 'ARB', 
                   'Alpha-Beta-blocker', 'Alpha-blocker', 'Sympath', 'Vasod'] # we donot include 'bpclass_none' as a action, because 000000000 means bpclass_none
"""

action_features = ['Diur', 'ACE', 'Beta-blocker', 'CCB', 'ARB'] 

combinations = list(itertools.product('01', repeat=len(action_features)))
actions = [''.join(i) for i in combinations]
print('len(actions) =', len(actions))
N_ACTIONS = len(actions) # number of actions = 512
action_code_to_index = {code: i for i, code in enumerate(actions)}
# print the first 5 action_code_to_index
for i in range(5):
    print(actions[i], action_code_to_index[actions[i]])


# build the action space for each state, assign the same action space to all states
actions_per_state = {}
for s in range(N_STATES):
    actions_per_state[s] = [i for i in range(N_ACTIONS)]
print('action_space for state 0:', actions_per_state[0])

## Calculate empirical estimates of P, R, C

In [ ]:
df = pd.read_csv(DATA)

In [ ]:
hba1c_discrete_code_dict = {'0': '0', '1': '0', 
                            '2': '1', '3': '1', 
                            '4': '2', '5': '2', 
                            '6': '3', '7': '3'}

# add the state and action code columns
action_code = []
state_code = []
for i in range(len(df)):
    row = df.iloc[i]
    s_code = ''
    a_code = ''
    for state_fea in state_features:
        code = str(row[state_fea])

        # merge 3 with 2 for sbp_discrete and TC_discrete
        if state_fea == 'sbp_discrete' and code == '3':
            code = '2'
        if state_fea == 'TC_discrete' and code == '3':
            code = '2'
        if state_fea == 'hdl_discrete' and code == '3':
            code = '2'

        # merge every 2 adjacent levels into 1 level for hba1c_discrete
        if state_fea == 'hba1c_discrete':
            code = hba1c_discrete_code_dict[code]
        
        s_code += code
    
    for action_fea in action_features:
        a_code += str(row[action_fea])
    
    action_code.append(a_code)
    state_code.append(s_code)

df['action_code'] = action_code
df['state_code'] = state_code
print('Finished adding action_code and state_code columns')

In [ ]:
#------------- calculate the empirical estimate of P, R, C
        
count_s_a = {} # count the number of times state s and action a appear in the dataset, sparse format
count_s_a_d = {} # count the number of times state s, action a, and next state s' appear in the dataset
sum_r_s_a = {} # sum of the reward of state s and action a
sum_c_s_a = {} # sum of the cost of state s and action a
visit_number = [] # number of visits for each patient

# loop through each patient in the dataset
for i in tqdm(range(100001, 110252)):
    df_patient = df[df['MaskID'] == i]

    if len(df_patient) > 0:
        visit_number.append(len(df_patient))

    # loop through each visit of the patient
    for j in range(len(df_patient)-1): # loop before last visit
        row = df_patient.iloc[j]
        s_code = row['state_code']
        a_code = row['action_code']
        ns_code = df_patient.iloc[j+1]['state_code']

        # convert from code to index
        s = state_code_to_index[s_code]
        a = action_code_to_index[a_code]
        s_ = state_code_to_index[ns_code]

        r = df_patient.iloc[j]['CVDRisk_feedback']
        c = df_patient.iloc[j]['sbp_feedback']

        if (s, a) not in count_s_a:
            count_s_a[(s, a)] = 1
            sum_r_s_a[(s, a)] = r 
            sum_c_s_a[(s, a)] = c
        else:
            count_s_a[(s, a)] += 1
            sum_r_s_a[(s, a)] += r
            sum_c_s_a[(s, a)] += c

        if (s, a, s_) not in count_s_a_d:
            count_s_a_d[(s, a, s_)] = 1
        else:
            count_s_a_d[(s, a, s_)] += 1

print('len(visit_number) =', len(visit_number))
print('averge visit_number =', sum(visit_number)/len(visit_number))

print('len(count_s_a) =', len(count_s_a))
print('len(count_s_a_d) =', len(count_s_a_d))
print('Finished counting by looping through the dataset')

In [ ]:
# calculate the sparsity of state-action pairs
print('Total possible state-action pairs =', N_STATES * N_ACTIONS)
print('seen state-action pairs =', len(count_s_a))
print('sparsity of state-action pairs =', 1 - len(count_s_a)/(N_STATES * N_ACTIONS))

In [ ]:
# calculate the empirical estimate of P, R, C using counts

# initialize R, C, P, NOT using sparse matrix format
R = {} # N_STATES * N_ACTIONS, dictionary of reward matrices, this is the CVDRisk empirical estimate based on entire dataset
C = {} # N_STATES * N_ACTIONS, dictionary of cost matrices, this is SBP empirical estimate based on entire dataset
P = {} # N_STATES * N_ACTIONS * N_STATES, dictionary of transition probability matrices, based on the entire dataset

for s in range(N_STATES):
    l = len(actions)
    R[s] = np.zeros(l)
    C[s] = np.zeros(l)
    P[s] = {}    
    for a in range(N_ACTIONS):
        C[s][a] = 0
        P[s][a] = np.zeros(N_STATES)
        R[s][a] = 0
print('Finished initializing R, C, P')

for (s,a) in count_s_a: # only calculate for the states and actions that appearedin the dataset, for efficiency

    # if s not in R:
    #     R[s] = {}
    #     C[s] = {}
    #     P[s] = {}
    
    # if a not in R[s]:
    #     R[s][a] = 0
    #     C[s][a] = 0
    #     P[s][a] = {}

    R[s][a] = sum_r_s_a[(s, a)]/max(count_s_a[(s, a)],1)
    C[s][a] = sum_c_s_a[(s, a)]/max(count_s_a[(s, a)],1)

for (s, a, s_) in count_s_a_d:
    P[s][a][s_] = count_s_a_d[(s, a, s_)]/max(count_s_a[(s, a)],1)

print('Finished calculating the empirical estimate of P, R, C')

#------------- check the sparsity of P, R, C
print('\nSparsity of P, R, C:')
print('P: {:.6f}% are non-zeros'.format(len(count_s_a_d)*100/(N_STATES*N_ACTIONS*N_STATES)))
print('R: {:.6f}% are non-zeros'.format(len(sum_r_s_a)*100/(N_STATES*N_ACTIONS)))
print('C: {:.6f}% are non-zeros'.format(len(sum_c_s_a)*100/(N_STATES*N_ACTIONS)))

In [ ]:
# # normalize R and C to the range [0, 1]

# # !!! we should not normalize C (sbp), becasue we need to satisfy the constraint that SBP in range 110-125
# # maybe no need to normalize R

# max_R = max([R[s][a] for s in R for a in range(N_ACTIONS)])
# max_C = max([C[s][a] for s in C for a in range(N_ACTIONS)])

# # max_R = max([R[s][a] for s in R for a in R[s]])
# # max_C = max([C[s][a] for s in C for a in C[s]])

# print('max_R =', max_R)
# print('max_C =', max_C)

# # for s in R:
# #     for a in range(N_ACTIONS):
# #         R[s][a] = (R[s][a])/(max_R)
# #         # C[s][a] = (C[s][a])/(max_C)


# #---------- assign average reward and cost to unseen state-action pairs

# # get the average of R and C
# R_sum = 0
# R_ct = 0
# C_sum = 0
# C_ct = 0
# for (s,a) in count_s_a:
#     R_sum += R[s][a]
#     R_ct += 1
#     C_sum += C[s][a]
#     C_ct += 1

# print('R_sum =', R_sum, 'R_ct =', R_ct)
# R_avg = R_sum/R_ct
# print('R_avg =', R_avg)

# print('C_sum =', C_sum, 'C_ct =', C_ct)
# C_avg = C_sum/C_ct
# print('C_avg =', C_avg)

# # assign the average to unseen state-action pairs
# for s in range(N_STATES):
#     for a in range(N_ACTIONS):
#         if (s,a) not in count_s_a:
#             R[s][a] = R_avg
#             C[s][a] = C_avg

In [ ]:
# # assign uniform probability to unseen state-action-state triples
# for s in range(N_STATES):
#     for a in range(N_ACTIONS):
#         if (s,a) not in count_s_a:
#             for s_ in range(N_STATES):
#                 P[s][a][s_] = 1/N_STATES

# # check the sparsity of P

# def check_sparsity_P(P):
#     count_nonzero_P = 0
#     for s in range(N_STATES):
#         for a in range(N_ACTIONS):
#             for s_ in range(N_STATES):
#                 if P[s][a][s_] != 0:
#                     count_nonzero_P += 1
#     print("Percentage of non-zero elements in P:", count_nonzero_P*100/(N_STATES*N_ACTIONS*N_STATES))

# check_sparsity_P(P)

In [ ]:
# # assign uniform probability to all state-action-state triples
# for s in range(N_STATES):
#     for a in range(N_ACTIONS):
#         for s_ in range(N_STATES):
#             P[s][a][s_] = 1/(N_STATES)

# check_sparsity_P(P)

## Check Init states

In [ ]:
def check_frequency(df, col_name):
    df = df[col_name]
    df = df.value_counts()
    print(df)
    print()

    # return the first index in the series
    return df.index[0]
    
# get the rows when the visit=='BLR' in df
df_blr = df[df['Visit']=='BLR']
INIT_STATES_LIST = df_blr['state_code'].unique() # we will sample uniformly from this list
print('len(INIT_STATES_LIST) =', len(INIT_STATES_LIST))

print('df_blr.shape =', df_blr.shape)
most_freq_blr_state = check_frequency(df_blr, 'state_code')
print('most_freq_blr_state =', most_freq_blr_state)
INIT_STATE_INDEX = state_code_to_index[most_freq_blr_state]
print('INIT_STATE_INDEX =', INIT_STATE_INDEX)

## Compute solution.pkl and baseline.pkl files

### Save the model settings

In [ ]:
# dump the model settings and parameters to a pickle file
with open('model.pkl', 'wb') as f:
    pickle.dump([NUMBER_SIMULATIONS, NUMBER_EPISODES, P, R, C, INIT_STATE_INDEX, INIT_STATES_LIST, state_code_to_index,
                CONSTRAINT, C_b, N_STATES, N_ACTIONS, actions_per_state, EPISODE_LENGTH, delta], f)

### Calculate the optimal policy

In [ ]:
import importlib
import sys
importlib.reload(sys.modules['UtilityMethods'])
from UtilityMethods import utils

util_methods_1 = utils(EPS, delta, M, P, R, C, INIT_STATE_INDEX, EPISODE_LENGTH, N_STATES, N_ACTIONS, actions_per_state, CONSTRAINT, C_b)

# constrained MDP, solve the optimal policy using LP
opt_policy_con, opt_value_LP_con, opt_cost_LP_con, opt_q_con = util_methods_1.compute_opt_LP_Constrained(0) 

# unconstrained = standard MDP, not used in DOPE
# opt_policy_uncon, opt_value_LP_uncon, opt_cost_LP_uncon, opt_q_uncon = util_methods_1.compute_opt_LP_Unconstrained(0) 

with open('solution.pkl', 'wb') as f:
    pickle.dump([opt_policy_con, opt_value_LP_con, opt_cost_LP_con, opt_q_con], f)

Have to deal with the sparsity of the P R C such that the above optimal solution is valid. For R amd C, assign the average value to unseen state-action pairs. For P, ???

### Calculate the baseline policy

In [ ]:
# baseline policy

util_methods_1 = utils(EPS, delta, M, P, R, C, INIT_STATE_INDEX, EPISODE_LENGTH, N_STATES, N_ACTIONS, actions_per_state, C_b, C_b)
policy_b, value_b, cost_b, q_b = util_methods_1.compute_opt_LP_Constrained(0)

with open('base.pkl', 'wb') as f:
    pickle.dump([policy_b, value_b, cost_b, q_b], f)

In [ ]:
print("opt_value_LP_con[0, 0] =",opt_value_LP_con[0, 0])
print("value_b[0, 0] =",value_b[0, 0])

### Decode calculated baseline policy

In [ ]:
# decode the policy_b [s, h, a]

for s in range(N_STATES):
    for h in range(EPISODE_LENGTH):
        for a in range(N_ACTIONS):
            if policy_b[s, h, a] != 0:
                print('policy_b[', s, ',', h, ',', a, '] =', policy_b[s, h, a], ', action_code =', actions[a])

## Other

### check state code frequency

In [ ]:
# check the frequency counts of hba1c_discrete in the dataset

# 'sbp_discrete','hba1c_discrete','TC_discrete','hdl_discrete','BMI_discrete'
check_frequency(df, 'sbp_discrete')
check_frequency(df, 'hba1c_discrete')
check_frequency(df, 'TC_discrete')
check_frequency(df, 'hdl_discrete')
check_frequency(df, 'BMI_discrete')

Frequency counts above show that:
1. we can merge the sbp_discrete 3 with 2. 
2. hba1c_discrete: merge nearby 2 levels into 1, e.g. 0,1 --> 0, 1,2 --> 1
3. TC_discrete: merge 3 with 2
4. hdl_discrete: merge 3 with 2
5. remove BMI_discrete since most are in 3

### BP_med frequency

In [ ]:
# check the sbp_feedback range in the df when prescribinng ACE, 
df_ace = df[df['ACE']==1]
print('df_ace.shape =', df_ace.shape)
print(min(df_ace['sbp_feedback']))
print(max(df_ace['sbp_feedback']))

# plot the distribution of sbp_feedback
plt.hist(df_ace['sbp_feedback'], bins=20)
plt.show()

In [ ]:
# check the sbp_feedback range in the df when prescribing ACE
df_ace_only = df[df['action_code']=='00000']
print('df_ace_only.shape =', df_ace_only.shape)
print(min(df_ace_only['sbp_feedback']))
print(max(df_ace_only['sbp_feedback']))

# plot the distribution of sbp_feedback
plt.hist(df_ace_only['sbp_feedback'], bins=20)
plt.show()

In [ ]:
# loop through each possible combination of actions 00000 - 11111 (total 32 combinations), check the corresponding range of sbp_feedback, and plot the distribution
# for i in range(32):
#     action = str(bin(i))[2:].zfill(5)
#     print(i, 'action =', action)
#     df_action = df[df['action_code']==action]
#     print('df_action.shape =', df_action.shape)
#     print(min(df_action['sbp_feedback']))
#     print(max(df_action['sbp_feedback']))
#     plt.hist(df_action['sbp_feedback'], bins=20)
#     plt.show()

## Check the state-action sparsity

In [ ]:
# loop though each row of df, bu

## Check correlation between state features

In [ ]:
state_features = ['sbp_discrete','hba1c_discrete','TC_discrete','hdl_discrete'] 

In [ ]:
sbp_discrete = df['sbp_discrete']
hba1c_discrete = df['hba1c_discrete']
TC_discrete = df['TC_discrete']
hdl_discrete = df['hdl_discrete']

# plot the correlation between every 2 features in state_features
import seaborn as sns
sns.pairplot(df[['sbp_discrete','hba1c_discrete','TC_discrete','hdl_discrete']], height=2)
plt.show()

No missing states.

In [ ]:
import seaborn as sns
sns.pairplot(df[['sbp','hba1c','TC','hdl']], height=2, markers='o')
plt.show()

In [ ]:
# create a 4x4 subplots, for every 2 features in state_features, plot the scatter plot
state_fea = ['sbp','hba1c','TC','hdl']
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(12,12))
for i in range(4):
    for j in range(4):
        # set the transparency to be 0.1
        axes[i,j].scatter(df[state_fea[i]], df[state_fea[j]], alpha=0.01)
        axes[i,j].set_xlabel(state_fea[i])
        axes[i,j].set_ylabel(state_fea[j])
plt.show()

In [ ]:
# check the correlation between every 2 features in state_features
df[['sbp_discrete','hba1c_discrete','TC_discrete','hdl_discrete']].corr()

In [ ]:
df[['sbp','hba1c','TC','hdl']].corr()

Plot the transition distribution

In [ ]:
# for each of the 5 features, plot the scatter plot between the feature and the its corresponding feedback
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,4))
for i in range(4):
    x_name = state_features[i]
    y_name = x_name.split('_')[0] + '_feedback_discrete'
    axes[i].scatter(df[x_name], df[y_name], alpha=0.01)
    axes[i].set_xlabel(x_name)
    axes[i].set_ylabel(y_name)
    
plt.show()


The sparsity comes from the transition of states. Not from the features.

Plot the 3D distribution point cloud

In [ ]:
# plot the 3D scatter plot between sbp, hba1c, and TC
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot( projection='3d')
# ax.scatter(df['sbp'], df['hba1c'], df['TC'], alpha=0.01)
ax.plot3D(df['sbp'], df['hba1c'], df['TC'], 'o', alpha=0.01)
ax.set_xlabel('sbp')
ax.set_ylabel('hba1c')
ax.set_zlabel('TC')
plt.show()


In [ ]:
df.info()